In [1]:
from bs4 import BeautifulSoup
import requests
import xml.etree.ElementTree as elemTree
import re
import time
import json

In [2]:
def download(method, url,param=None, data=None, timeout=1, maxretries=3, headers={"user-agent":None}):
    try:
        resp = requests.request(method, url, params=param, data=data, headers=headers, verify=False)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        #print(e.code)
        #print(e.reason)
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            print(maxretries)
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [9]:
def toJson(data):
    with open('law_data.json', 'w', encoding='utf-8') as file :
        json.dump(data, file, ensure_ascii=False, indent='\t')

In [4]:
# 법 리스트 뽑아내기
def getLawList(url):
    lawList = list()
    html = download('get', url)
    dom = BeautifulSoup(html.text, 'lxml')
    lists = dom.select('tbody > tr > .subject')
    
    for _ in lists:
         lawList.append(_)
            
    return lawList

In [5]:
# 법의 각 조 이름과 해당 내용 뽑아내기
def getJomoonContents(name):
    url = 'https://www.law.go.kr/DRF/lawService.do?OC=qpsdhkwmf12&target=law&type=xml&LM=' + str(name)
    xml = download('get', url)
    tree = elemTree.fromstring(xml.text)
    jomoonList = tree.findall('조문/조문단위')
    
    jomoon = list()
    
    for _ in jomoonList:
        if _.get('조문키')[-1] != '0':
            contents = ''
            
            # 조 이름 뽑기 위함
            jo = _.find('조문내용').text
            end = jo.find(')')
    
            contents += _.find('조문내용').text.replace(jo[:end+1], '')

            if _.find('항') is not None:
                for _ in _.findall('항'):
                    if _.find('항내용') is not None:
                        contents += _.find('항내용').text.strip() + '\n'

                    if _.find('호') is not None:
                        for _ in _.findall('호'):
                            contents += _.find('호내용').text.strip() + '\n'

                            if _.find('목') is not None:
                                for _ in _.findall('목'):
                                    contents += _.find('목내용').text.strip() + '\n'
                                
            jomoon.append((jo[:end+1], contents))
        
    return jomoon

In [10]:
law_list = list()

for page in range(1,23):
    url = 'http://www.ccn.go.kr/lawlist/lawlist.ccn?gSiteCode=2&nMenuCode=19&gMenuCode=5&page=' + str(page)
    lawList = getLawList(url)
    
    for i,_ in enumerate(lawList, page*10-9):
        print(i, _.text)
        law = dict()
        law['index'] = i
        law['name'] = _.text
    
        jomoon = getJomoonContents(_.text)
        jomoon_list = list()
        for _ in jomoon:
            jomoon = dict()
            
            jomoon['joNum'] = _[0]
            jomoon['contents'] = _[1]
            jomoon_list.append(jomoon)

        law['jomoon'] = jomoon_list
        law_list.append(law)
    
toJson(law_list)

1 가맹사업거래의 공정화에 관한 법률


C:\Users\Wonsun Yoo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2 가축전염병예방법


C:\Users\Wonsun Yoo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


3 가축전염병예방법 시행규칙


...

C:\Users\Wonsun Yoo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


212 형법


C:\Users\Wonsun Yoo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


213 화장품법


C:\Users\Wonsun Yoo\Anaconda3\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
